### Assignment 1

First step is to import libraries and complete other pre-processing steps that will set up our data to input into a model for our first model that predicts if a given user will play a given game.

In [101]:
import gzip
from collections import defaultdict
import numpy as np
import pandas as pd
import string
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity


import scipy.spatial as sp
import scipy.sparse

In [102]:
li = []
def readJSON(path):
    for l in gzip.open(path, 'rt'):
        d = eval(l)
        u = d['userID']
        try:
            g = d['gameID']
        except Exception as e:
            g = None
        yield u,g,d
        
gameCount = defaultdict(int)
totalPlayed = 0

for user,game,_ in readJSON("train.json.gz"):
    gameCount[game] += 1
    totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

c = 0
X_train = []
X_val = []
userGames = defaultdict(set)
gameUsers = defaultdict(set)
userHours = defaultdict(list)
gameHours = defaultdict(list)
received_free = defaultdict(list)
users = set()
games = set()
for user, game, d in readJSON('train.json.gz'):
    if d['hours_transformed'] > 8.5:
        continue
    else:
        userGames[user].add(game)
        gameUsers[game].add(user)
        users.add(user)
        games.add(game)
        userHours[user].append(d['hours_transformed'])
        gameHours[game].append(d['hours_transformed'])
        if 'compensation' in d.keys():
            received_free[game].append(1)
        else:
            received_free[game].append(0)
        c = np.random.choice([1, 0], p=[0.8, 0.2])
        if c == 1:
            X_train.append(d)
        else:
            X_val.append(d)
    
X_train = pd.DataFrame(X_train)
X_train['y'] = np.array([1]*X_train.shape[0])
X_val = pd.DataFrame(X_val)
X_val['y'] = np.array([1]*X_val.shape[0])

def user_didnt_play(uID):
    g = list(userGames[uID])
    g_prime = games.copy()
    for i in g:
        g_prime.remove(i)
    return np.random.choice(list(g_prime))

negative = []
for i in list(X_train['userID']):
    negative.append({'userID': i, 'gameID': user_didnt_play(i), 'y': 0})
neg = pd.DataFrame(negative)
X_train = X_train.append(neg).reset_index(drop=True)

negative = []
for i in list(X_val['userID']):
    negative.append({'userID': i, 'gameID': user_didnt_play(i), 'y': 0})
    
neg = pd.DataFrame(negative)
X_val = X_val.append(neg).reset_index(drop=True)

X_train = X_train.sample(frac=1).reset_index(drop=True)
X_val = X_val.sample(frac=1).reset_index(drop=True)

totGames = list(totGames)
totGames = {totGames[x]:x  for x in range(len(totGames))}

X_train['hours_transformed'] = X_train['hours_transformed'].fillna(0)
X_val['hours_transformed'] = X_val['hours_transformed'].fillna(0)

In [103]:
#received_free
X_train['compensation'] = X_train['compensation'].fillna(0)
X_val['compensation'] = X_val['compensation'].fillna(0)
X_train['compensation'].value_counts()

0                            267257
Product received for free      2191
Name: compensation, dtype: int64

In [110]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalPlayed/1.5:
        break

def make_features(data):
    features = []
    for i in range(data.shape[0]):
        if i % 1000 == 0:
            print('Working on ' + str(i) + ' now...')
        jac_vals = []
        user, game = data['userID'][i], data['gameID'][i]
        gUsers = gameUsers[game]
        uGames = userGames[user]
        
        for x in uGames:
            if x == game:
                continue
            jac_vals.append(Jaccard(gUsers, gameUsers[x]))
                   
        if len(jac_vals) == 0:
            jac_max = 0
        else:
            jac_max = max(jac_vals)
            
            
        pop_orNot = float(game in return1)
        
        if len(userHours[user]) == 0:
            avg_userHours = 0
        else:
            avg_userHours = np.median(userHours[user])
            
        if len(gameHours[game]) == 0:
            avg_gameHours = 0
        else:
            avg_gameHours = np.median(gameHours[game])
        
        if len(received_free[game]) == 0:
            reFree = 0
        else:
            reFree = np.sum(received_free[user])

        features.append([jac_max, pop_orNot, avg_userHours, reFree])
            
    return features

In [111]:
feats = make_features(X_train)

Working on 0 now...
Working on 1000 now...
Working on 2000 now...
Working on 3000 now...
Working on 4000 now...
Working on 5000 now...
Working on 6000 now...
Working on 7000 now...
Working on 8000 now...
Working on 9000 now...
Working on 10000 now...
Working on 11000 now...
Working on 12000 now...
Working on 13000 now...
Working on 14000 now...
Working on 15000 now...
Working on 16000 now...
Working on 17000 now...
Working on 18000 now...
Working on 19000 now...
Working on 20000 now...
Working on 21000 now...
Working on 22000 now...
Working on 23000 now...
Working on 24000 now...
Working on 25000 now...
Working on 26000 now...
Working on 27000 now...
Working on 28000 now...
Working on 29000 now...
Working on 30000 now...
Working on 31000 now...
Working on 32000 now...
Working on 33000 now...
Working on 34000 now...
Working on 35000 now...
Working on 36000 now...
Working on 37000 now...
Working on 38000 now...
Working on 39000 now...
Working on 40000 now...
Working on 41000 now...
Worki

In [112]:
model1 = LogisticRegression(C=10, fit_intercept=True)
model1.fit(feats, X_train['y'])

LogisticRegression(C=10)

In [113]:
preds = model1.predict(make_features(X_val))
preds

Working on 0 now...
Working on 1000 now...
Working on 2000 now...
Working on 3000 now...
Working on 4000 now...
Working on 5000 now...
Working on 6000 now...
Working on 7000 now...
Working on 8000 now...
Working on 9000 now...
Working on 10000 now...
Working on 11000 now...
Working on 12000 now...
Working on 13000 now...
Working on 14000 now...
Working on 15000 now...
Working on 16000 now...
Working on 17000 now...
Working on 18000 now...
Working on 19000 now...
Working on 20000 now...
Working on 21000 now...
Working on 22000 now...
Working on 23000 now...
Working on 24000 now...
Working on 25000 now...
Working on 26000 now...
Working on 27000 now...
Working on 28000 now...
Working on 29000 now...
Working on 30000 now...
Working on 31000 now...
Working on 32000 now...
Working on 33000 now...
Working on 34000 now...
Working on 35000 now...
Working on 36000 now...
Working on 37000 now...
Working on 38000 now...
Working on 39000 now...
Working on 40000 now...
Working on 41000 now...
Worki

array([0, 0, 1, ..., 1, 1, 1])

In [114]:
np.mean(np.array(preds) == X_val['y'])

0.7696280074576071

In [115]:
p = open("predictions_Played.txt", 'w')
d = []
for l in open("pairs_Played.txt"):
    if l.startswith("userID"):
    #header
        p.write(l)
        continue
    u,g = l.strip().split('-')
    d.append({'userID': u, 'gameID': g})
    
d = pd.DataFrame(d)
d['preds'] = model1.predict(make_features(d))

for i in range(d.shape[0]):
    p.write(d['userID'][i] + '-' + d['gameID'][i] + ',' + str(d['preds'][i]) + '\n')

p.close()

Working on 0 now...
Working on 1000 now...
Working on 2000 now...
Working on 3000 now...
Working on 4000 now...
Working on 5000 now...
Working on 6000 now...
Working on 7000 now...
Working on 8000 now...
Working on 9000 now...
Working on 10000 now...
Working on 11000 now...
Working on 12000 now...
Working on 13000 now...
Working on 14000 now...
Working on 15000 now...
Working on 16000 now...
Working on 17000 now...
Working on 18000 now...
Working on 19000 now...


The next part is to build a model that predicts the category of the game based on textual features of the review.

In [52]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
from scipy.sparse import lil_matrix

def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)
        
def readJSON(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        d = eval(l)
        u = d['userID']
        yield u,d

In [53]:
data = []

for d in readGz("train_Category.json.gz"):
    data.append(d)

Ntrain = (9*len(data))//10
dataTrain = data[:Ntrain]

data_test = []
users_and_reviewID = []
for d in readGz("test_Category.json.gz"):
    data_test.append(d)
    users_and_reviewID.append((d['userID'], d['reviewID']))

In [54]:
sp = set(string.punctuation)
wordCount = defaultdict(int)
for d in data:
    r = ''.join([c for c in d['text'].lower() if not c in sp])
    tokens = r.split()
    for w in tokens:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
counts[:10]

[(576212, 'the'),
 (336527, 'and'),
 (323655, 'a'),
 (309606, 'to'),
 (259975, 'game'),
 (240526, 'of'),
 (220624, 'is'),
 (212817, 'you'),
 (207779, 'i'),
 (202640, 'it')]

In [63]:
NW = 3500
words = [x[1] for x in counts[:NW]]

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['text'].lower() if not c in sp])
    tokens = r.split()
    for w in tokens:
        if w in wordSet:
            feat[wordId[w]] += 1
                
    feat.append(1)
    feat.append(len(tokens))
    feat.append(float(datum['early_access']))
    return feat

nf = len(feature(data[0]))

#Sparse matrix
X = lil_matrix((len(data), nf))

for i in range(len(data)):
    if not (i % 1000):
        print(i)
    x = feature(data[i])
    for j in range(nf):
        if x[j]:
            X[i,j] = x[j]

y = [d['genreID'] for d in data]

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [64]:
Xtrain = X[:Ntrain]
ytrain = y[:Ntrain]
Xvalid = X[Ntrain:]
yvalid = y[Ntrain:]
mod = linear_model.LogisticRegression(C=10, max_iter=10000)
mod.fit(Xtrain, ytrain)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=10, max_iter=10000)

In [65]:
pred = mod.predict(Xvalid)
correct = pred == yvalid
sum(correct) / len(correct)

0.7209142857142857

In [66]:
nf = len(feature(data_test[0]))
X = lil_matrix((len(data_test), nf))

for i in range(len(data_test)):
    if not (i % 1000):
        print(i)
    x = feature(data_test[i])
    for j in range(nf):
        if x[j]:
            X[i,j] = x[j]
            
X

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


<10000x3502 sparse matrix of type '<class 'numpy.float64'>'
	with 414645 stored elements in List of Lists format>

In [67]:
predictions = open("predictions_Category.txt", 'w')
categories = {0:  'Action', 1: 'Strategy', 2: 'RPG', 3: 'Adventure', 4: 'Sports'}
predictions.write('userID-reviewID,prediction\n')
    
p = mod.predict(X)
for i in range(len(p)):
    predictions.write(users_and_reviewID[i][0] + '-' + users_and_reviewID[i][1] + ',' + str(p[i]) + "\n")
        
predictions.close()

In [60]:
data_test

[{'hours': 3.2,
  'early_access': False,
  'date': '2016-03-17',
  'reviewID': 'r23357682',
  'text': "You shouldn't really go into this game looking for a serious experience, this game is a joke and a shallow fun experience and that's great. There is no need to pretend that its any deeper than that. Its nice that the game has a couple of jokes beyond the obvious one built into it and they unfold as you play game giving a bit more longevity. But at the end of the day, its just a fun flash game and even at the price of a dollar I wouldn't buy for myself when I can play games like Rebuild, Sonny and others for free and they offer really deep gaming experiences.\nIf you want to read more about the game in an totally ironic way you can find my review here:\n.",
  'userID': 'u62606497'},
 {'hours': 10.0,
  'early_access': False,
  'date': '2017-01-06',
  'reviewID': 'r31389322',
  'text': "The gameplay is nice and all but this game is terrible unoptimized no matter what excuse this communit